In [1]:
import os
from openai import AzureOpenAI
import json
os.environ["USE_SCHEMA_BATCH_MODE"] = "false"
from Schema import ExtractedData
import pandas as pd

# you need to reload the notebok after changing the Schema
# simply reruning this cell will not work

Schema: Interactive Mode


In [2]:
df = pd.read_csv("data.csv")
# make sure id is string
df["id"] = df["id"].astype(str)

In [3]:
with open('key.txt', 'r') as file:
    key = file.read().replace('\n', '')

with open('endpoint.txt', 'r') as file:
    endpoint = file.read().replace('\n', '')

client = AzureOpenAI(api_key=key,
                     api_version="2024-10-21",
                        azure_endpoint=endpoint)
del key, endpoint

deployment_name = "gpt-4o-2"

In [4]:
system_prompt = "You are an expert in extracting information from recepies."
base_prompt = '''Read the recipe and fill out the ExtractedData accordingly. Also give the recipe a funny pretentious name.
'''
# write to json file
with open('prompt.json', 'w') as f:
    json.dump({"system_prompt": system_prompt, "base_prompt": base_prompt}, f, ensure_ascii=False, indent=4)

In [5]:
id = "90"
# find df with id 90
df_text = df[df['id'] == id]["text"].values[0]
prompt = base_prompt + df_text
print(prompt)

Read the recipe and fill out the ExtractedData accordingly. Also give the recipe a funny pretentious name.
Preheat the oven to 375 degrees F (190 degrees C). Grease a 12-cup muffin pan.
Mix flour, baking powder, baking soda, and salt in a medium bowl.
Beat together sugar, butter, and eggs in a large bowl until smooth. Mix in vanilla. Stir in apples, then gradually blend in flour mixture. Spoon batter into the prepared muffin pan.
Make topping: Mix brown sugar, flour, and cinnamon in a small bowl. Cut in butter until mixture resembles coarse crumbs. Sprinkle over each muffin top.
Bake in the preheated oven until a toothpick inserted in the center of a muffin comes out clean, about 20 minutes. Let sit 5 minutes before transferring muffins to a wire rack to cool.


In [6]:
response = client.beta.chat.completions.parse(
            model = deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            max_tokens=1500,
            response_format=ExtractedData,
            temperature=0.2
        )

In [7]:
cost = (response.usage.completion_tokens / 1000 * 0.015 + response.usage.prompt_tokens / 1000 * 0.005) # costs depend on model and are 50% in batch mode
reason = response.choices[0].finish_reason
print(f"Finish Reason: {reason}")
print(f"Prompt Tokens: {response.usage.prompt_tokens}")
print(f"Completion Tokens: {response.usage.completion_tokens}")
print(f"Cost: ${cost:.4f}")
txt_response = response.choices[0].message.content
print(txt_response)

Finish Reason: stop
Prompt Tokens: 510
Completion Tokens: 74
Cost: $0.0037
{"pretentious_recipy_name":"Artisanal Apple Infusion Muffin Masterpiece","is_vegan":false,"oven_instructions":{"preheat_temperature_celcius":190,"time_in_oven_minutes":20},"recipe_type":"dessert","necessary_utensils":["oven","muffin_pan","bowl","wire_rack"]}


In [8]:
some_json = json.loads(txt_response)
print(json.dumps(some_json, indent=4, sort_keys=True, ensure_ascii=False))

{
    "is_vegan": false,
    "necessary_utensils": [
        "oven",
        "muffin_pan",
        "bowl",
        "wire_rack"
    ],
    "oven_instructions": {
        "preheat_temperature_celcius": 190,
        "time_in_oven_minutes": 20
    },
    "pretentious_recipy_name": "Artisanal Apple Infusion Muffin Masterpiece",
    "recipe_type": "dessert"
}


In [9]:
# load response into pydantic model
knee_mri_report = ExtractedData.model_validate(some_json)

In [10]:
print(knee_mri_report)

pretentious_recipy_name='Artisanal Apple Infusion Muffin Masterpiece' is_vegan=False oven_instructions=OvenInstructions(preheat_temperature_celcius=190, time_in_oven_minutes=20) recipe_type=<RecipeType.DESSERT: 'dessert'> necessary_utensils=[<KitchenUtensil.oven: 'oven'>, <KitchenUtensil.muffin_pan: 'muffin_pan'>, <KitchenUtensil.bowl: 'bowl'>, <KitchenUtensil.wire_rack: 'wire_rack'>]
